In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float})    
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len (f)>0:
        df=df.drop(row.Index)


In [199]:
import math
import numpy as np
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float})
homef=50
w=0
df.sort_values(by='Date',inplace=True)
l=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=50
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral==True:
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!=True:
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!=True:
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())
eloLeague=Elo(k=20)
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
for row in df.itertuples():
    if row.score1>row.score2 or row.score1==row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=50
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            else:
                l+=1
            winnerhome=False
        else:
            hfa=0
            if row.VenueType!='N':
                hfa=50
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            else:
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=50
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            else:
                l+=1
            winnerhome=True
        else:
            winnerhome=False
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)

In [202]:
df[df.VenueType=='N']

,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
23,Sun,7/17/1983,N,Michigan Panthers,22-24,L,,"Denver, CO",Mile High Stadium,50906,Philadelphia Stars,1983.0,24,22
107,Sun,7/17/1983,N,Philadelphia Stars,24-22,W,,"Denver, CO",Mile High Stadium,50906,Michigan Panthers,1983.0,22,24
277,Sun,7/15/1984,N,Arizona Wranglers,23-3,W,,"Tampa, FL",Tampa Stadium,52661,Philadelphia Stars,1984.0,3,23
278,Sat,7/21/1984,N,Tampa Bay Bandits,24-21,W,,"London, England",Wembley Satdium,21000,Philadelphia Stars,1984.0,21,24
387,Sat,7/21/1984,N,Philadelphia Stars,21-24,L,,"London, England",Wembley Stadium,21000,Tampa Bay Bandits,1984.0,24,21
601,Sun,7/15/1984,N,Philadelphia Stars,3-23,L,,"Tampa, FL",Tampa Stadium,52661,Arizona Wranglers,1984.0,23,3
737,Sun,7/14/1985,N,Oakland Invaders,28-24,W,,"East Rutherford, NJ",Giants Stadium,49263,Baltimore Stars,1985.0,24,28
825,Sun,7/14/1985,N,Baltimore Stars,24-28,L,,"East Rutherford, NJ",Giants Stadium,49263,Oakland Invaders,1985.0,28,24


In [197]:
df.to_csv("usfl.csv")

In [189]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

Birmingham Stallions 1906.6106290160808
New Jersey Generals 1841.8974560052795
 Philadelphia Stars 1839.8738173846752
 Baltimore Stars 1798.596724350298
Baltimore Stars 1786.8492627346118
 New Jersey Generals 1775.5941151224692
Philadelphia Stars 1763.3752258393365
New Orleans Breakers 1747.2820878331133
 Arizona Wranglers 1711.0168451011143
 Oakland Invaders 1704.7264295401953
Memphis Showboats 1670.8060227027847
 Memphis Showboats 1657.5690478656834
Oakland Invaders 1653.5313984925344
 Birmingham Stallions 1642.0268778211978
Denver Gold 1621.1583572704276
 Boston Breakers 1588.7632392165754
Boston Breakers 1581.720443794869
Arizona Wranglers 1578.977727406429
 New Orleans Breakers 1578.2909971842223
Jacksonville Bulls 1559.7685206789806
 Houston Gamblers 1557.669798176626
 Jacksonville Bulls 1543.7423329126354
Houston Gamblers 1531.060492262618
Arizona Outlaws 1514.496458959265
 Arizona Outlaws 1472.683801598229
 Tampa Bay Bandits 1469.0358223112225
Chicago Blitz 1419.771110322741
 M